In [ ]:
# To generate a PDF out of this, run:
# jupyter nbconvert --to pdf --template notebooks/hidecode notebooks/top5.ipynb

In [11]:
from pathlib import Path

import pandas as pd
from IPython.display import display, Markdown


raw_data_dir = Path("../../raw_data")
reports_dir = Path("../../reports")
season_id = 25

season_raw_data_dir = raw_data_dir / f"season{season_id}"
season_reports_dir = reports_dir / f"season{season_id}"

pl = pd.read_csv(season_raw_data_dir / "player_logs.csv")

stats_enabled = [
    "pts",
    "fgm",
    "fga",
#     "fg%",
    "3pm",
    "3pa",
#     "3p%",
    "ftm",
    "fta",
#     "ft%",
    "oreb",
    "dreb",
    "reb",
    "ast",
    "stl",
    "tov",
    "blk",
    "blka",
    "fc",
    "fd",
    "pir",
]

stats_groups = [
    [
        "Performance Index Rating",
        ["pir", "pts", "reb", "ast", "tov", "stl", "blk", "fc"],
        [False, False, False, False, True, False, False, True],
    ],
    [
        "Points",
        ["pts", "fgm", "fga", "3pm", "3pa", "ftm", "fta"],
        [False, False, True, False, True, False, True],
    ],
    
    [
        "Rebounds",
        ["reb", "oreb", "dreb"],
        [False, False, False],
    ],
    [
        "Offensive Rebounds",
        ["oreb", "dreb", "reb"],
        [False, False, False],
    ],
    [
        "Assists",
        ["ast", "tov"],
        [False, True],
    ],
    [
        "Steals",
        ["stl", "tov", "fc"],
        [False, True, True],
    ],
    [
        "Blocks",
        ["blk",],
        [False,],
    ],
    [
        "Turnovers",
        ["tov",],
        [False,],
    ],
    [
        "Fouls Committed",
        ["fc", "fd",],
        [False, True],
    ],
]


In [12]:
def fix_column_names(name):
    if name.endswith("_name"):
        name = name[:-5]
    return name.upper()


player_totals = []
for (team_id, player_id, player_gender), group in pl.groupby(["team_id", "player_id", "player_gender"]):
    pt = {
        "team_id": team_id,
        "team_name": group["team_name"].iloc[0],
        "player_id": player_id,
        "player_gender": player_gender,
        "player_name": group["player_name"].iloc[0],
        **{s: group[s].sum() for s in stats_enabled},
        **{f"{s}_avg": group[s].mean() for s in stats_enabled},
    }
    player_totals.append(pt)

team_totals = []
for team_id, group in pl.groupby("team_id"):
    tt = {
        "team_id": team_id,
        "team_name": group["team_name"].iloc[0],
        **{s: group[s].sum() for s in stats_enabled},
        **{f"{s}_avg": group[s].mean() for s in stats_enabled},
    }
    team_totals.append(tt)
    
ptdf = pd.DataFrame(player_totals)
ttdf = pd.DataFrame(team_totals)

for title, sort_fields, sort_ascending in stats_groups:
    display(Markdown(f"## {title}"))
    
    teams_sorted_by_field = ttdf.sort_values(
        sort_fields,
        ascending=sort_ascending,
    )
    top8_by_field = teams_sorted_by_field[["team_name", *sort_fields]].head(8)
    top8_by_field.reset_index(drop=True, inplace=True)
    top8_by_field.index += 1
    top8_by_field.columns = map(fix_column_names, top8_by_field.columns)
    display(top8_by_field)

    for gender in ("f", "m"):
        this_gender = ptdf["player_gender"] == gender
        sorted_by_field = ptdf[this_gender].sort_values(
            sort_fields,
            ascending=sort_ascending,
        )
        top5_by_field = sorted_by_field[["player_name", "team_name", *sort_fields]].head(5)
        top5_by_field.reset_index(drop=True, inplace=True)
        top5_by_field.index += 1
        top5_by_field.columns = map(fix_column_names, top5_by_field.columns)
        display(top5_by_field)


## Performance Index Rating

,TEAM,PIR,PTS,REB,AST,TOV,STL,BLK,FC
1,Rockets,93,70,42,20,3,10,0,14
2,Burritos,70,50,36,14,7,5,3,4
3,Lost Angels,69,56,37,17,8,2,3,5
4,Lions,30,39,33,8,10,4,0,7
5,Camden Hells,27,34,25,8,6,5,4,3
6,Green Machine,21,34,31,4,9,6,1,6
7,North L Lions,10,29,18,9,8,7,0,8
8,Hot Shots,9,22,20,5,17,3,1,9


,PLAYER,TEAM,PIR,PTS,REB,AST,TOV,STL,BLK,FC
1,Janis,Rockets,13,9,3,7,0,2,0,3
2,Mary,Burritos,10,9,3,0,0,1,0,0
3,Sophie,Lost Angels,10,6,4,3,1,0,1,0
4,Daria,Lost Angels,8,9,3,1,0,1,0,0
5,Weronika,Hot Shots,6,0,2,3,1,0,0,0


,PLAYER,TEAM,PIR,PTS,REB,AST,TOV,STL,BLK,FC
1,Wouter,Burritos,27,23,11,2,2,0,3,3
2,Marc,Rockets,23,17,5,5,0,3,0,2
3,Rolands,Lost Angels,23,10,15,4,2,1,2,3
4,Darnell,Rockets,19,17,7,3,0,0,0,1
5,AlexGM,Green Machine,17,7,14,2,0,3,1,4


## Points

...

,PLAYER,TEAM,TOV
1,Harrison38,Hot Shots,6
2,Diego,Lions,4
3,Tom,Burritos,4
4,Ahmed,Green Machine,3
5,Sharad,Hot Shots,3


## Fouls Committed

,TEAM,FC,FD
1,Rockets,14,9
2,Hot Shots,9,16
3,North L Lions,8,0
4,Lions,7,0
5,Green Machine,6,0
6,Lost Angels,5,0
7,Burritos,4,0
8,Camden Hells,3,0


,PLAYER,TEAM,FC,FD
1,Patricia,Rockets,4,0
2,Janis,Rockets,3,2
3,Kata,Lions,2,0
4,Armelle,Burritos,1,0
5,Steph-Kansas,Rockets,1,0


,PLAYER,TEAM,FC,FD
1,Harrison38,Hot Shots,5,7
2,AlexGM,Green Machine,4,0
3,Gus,Lions,4,0
4,Wouter,Burritos,3,0
5,Rolands,Lost Angels,3,0
